# Feature Engineering

This notebook is a natural continuation of the exploratory data analysis in the Exploratory_Data_Analysis notebook.

Key findings from the EDA were:
- Missing values are relatively frequent, so a solution for nulls should be considered
- The two cities that the measurements are taken from have:
    - Different (hard to measure how different without more data..) climates according to the weather data
    - Have different train and test time measurement periods. And the test period immediately follows the training period for each city
        - This means the problem can be considered as a forecasting one, for each city. 
        - Initial feelings are that the model should condition for each city.
            
- A quick peek of raw feature predictive power suggest that weekofyear, humidity, rain and temperature could be important in the model.

The purpose of this notebook will be to engineer features from the raw data to use in the final model. This will be a combination of:
- Intuition (e.g. time/seasonal related feature construction)
- Using SKLearn to explore features derived from PCA, polynomial, 'fromModel' and 'KBest' approaches.
- Construct 'feature sets' from the analysis, to be used in model exploration. 


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_context('talk')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")